In [ ]:
import requests
import os
from pathlib import Path
from dotenv import load_dotenv
import zipfile
import subprocess

# Load .env variables
load_dotenv(dotenv_path="configs/.env")

GITHUB_REPO = "QuAOS-Lab/quaos"
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
PROFILING_FOLDER = Path() / "data/profiling"

def get_recent_workflow_runs(repo, token, limit=5):
    """Fetch the most recent completed workflow runs."""
    url = f"https://api.github.com/repos/{repo}/actions/runs"
    headers = {
        "Authorization": f"token {token}",
        "Accept": "application/vnd.github.v3+json"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    runs = response.json()["workflow_runs"]
    
    # Filter only completed runs (success, failure, etc.)
    completed_runs = [run for run in runs if run["status"] == "completed"]
    
    return completed_runs[:limit]  # Return only the latest 'limit' runs

def download_artifact(artifact, repo, token):
    """Download and save an artifact as a ZIP file."""
    url = artifact["archive_download_url"]
    headers = {
        "Authorization": f"token {token}",
        "Accept": "application/vnd.github.v3+json"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    
    PROFILING_FOLDER.mkdir(parents=True, exist_ok=True)
    
    zip_path = PROFILING_FOLDER / f"{artifact['name']}.zip"
    with open(zip_path, "wb") as file:
        file.write(response.content)
    
    print(f"Downloaded {artifact['name']}.zip")
    return zip_path

def extract_zip(zip_path, extract_to="."):
    """Extract a ZIP file."""
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Extracted {zip_path}")

def display_profiling_results(profiling_file):
    """Open profiling results in SnakeViz and terminate the script when the page is closed."""
    print(f"\nOpening profiling results: {profiling_file}")
    subprocess.run(["snakeviz", profiling_file])
    print("SnakeViz closed. Exiting script.")
    exit()

def get_artifacts(run_id, repo, token):
    """Fetch artifacts from a specific workflow run."""
    url = f"https://api.github.com/repos/{repo}/actions/runs/{run_id}/artifacts"
    headers = {
        "Authorization": f"token {token}",
        "Accept": "application/vnd.github.v3+json"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()["artifacts"]


if __name__ == "__main__":
    if not GITHUB_TOKEN:
        raise Exception("GITHUB_TOKEN environment variable not set")

    # Fetch the last 5 completed workflow runs
    recent_runs = get_recent_workflow_runs(GITHUB_REPO, GITHUB_TOKEN)
    
    if not recent_runs:
        raise Exception("No completed workflow runs found.")
    
    # Display recent workflow runs
    print("\nRecent Completed Workflow Runs:")
    for i, run in enumerate(recent_runs):
        print(f"{i + 1}. Run ID: {run['id']}, Status: {run['conclusion']}, Created: {run['created_at']}")

    # Let the user pick a workflow run
    selected_index = int(input("\nSelect a workflow run (1-5): ")) - 1
    if selected_index not in range(len(recent_runs)):
        raise Exception("Invalid selection!")

    selected_run = recent_runs[selected_index]
    print(f"\nFetching artifacts from Run ID: {selected_run['id']}...\n")

    # Fetch artifacts from the selected run
    artifacts = get_artifacts(selected_run["id"], GITHUB_REPO, GITHUB_TOKEN)

    if not artifacts \
        or "profiling-results" not in [artifact["name"] for artifact in artifacts]:
        raise Exception("No profiling artifacts found for the selected run.")

    # Download and extract the selected artifact
    selected_artifact = next(artifact for artifact in artifacts if artifact["name"] == "profiling-results")
    zip_path = download_artifact(selected_artifact, GITHUB_REPO, GITHUB_TOKEN)
    extract_zip(zip_path, extract_to=PROFILING_FOLDER)

    # If it's profiling data, display it
    if selected_artifact["name"] == "profiling-results":
        profiling_file = PROFILING_FOLDER / "profile.out"
        display_profiling_results(profiling_file)
